# **Install the required the libaries**

In [ ]:
%pip install llama_index ftfy regex tqdm
%pip install -U openai-whisper
%pip install git+https://github.com/openai/CLIP.git
%pip install torch torchvision
%pip install matplotlib scikit-image
%pip install lancedb
%pip install moviepy
%pip install pytube
%pip install pydub
%pip install SpeechRecognition
%pip install ffmpeg-python
%pip install soundfile

# **Import the required Libaries**

In [ ]:
from moviepy.editor import VideoFileClip
from pathlib import Path
import speech_recognition as sr
from pytube import YouTube
from pprint import pprint
from PIL import Image
import matplotlib.pyplot as plt

In [ ]:
video_url="https://youtu.be/Xr8QZHQm8PA?si=uhh0r0pGb6ugr4Rn"
output_video_path = "/content/video_data/"
output_folder = "/content/mixed_data/"
output_audio_path = "/content/data/output_audio.wav"

In [ ]:
filepath=output_video_path + "input_vid.mp4"

In [ ]:
filepath

# **Processing on video**

In [ ]:
def plot_images(images_path):
  images_shown = 0
  plt.figure(figsize=(16, 9))
  for img_path in images_path:
        if os.path.isfile(img_path):
            image = Image.open(img_path)

            plt.subplot(2, 3, images_shown + 1)
            plt.imshow(image)
            plt.xticks([])
            plt.yticks([])

            images_shown += 1
            if images_shown >= 5:
                break


In [ ]:
from pytube import YouTube
def download_video(url,output_path):
  yt = YouTube(url)
  metadata = {"Author": yt.author, "Title": yt.title, "Views": yt.views}
  yt.streams.get_highest_resolution().download(
        output_path=output_path, filename="input_vid.mp4"
    )
  return metadata

In [ ]:
from moviepy.editor import VideoFileClip
import os
def video_to_images(video_path,output_flder):
  clip=VideoFileClip(video_path)
  clip.write_images_sequence(
      os.path.join(output_folder,"frame%04d.jpg"),fps=0.2
  )

In [ ]:
def video_to_audio(video_path,output_audio_path):
  clip=VideoFileClip(video_path)
  audio=clip.audio
  audio.write_audiofile(output_audio_path)


In [ ]:
def audio_to_text(audio_path):
  recognizer=sr.Recognizer()
  audio=sr.AudioFile(audio_path)

  with audio as source:
    audio_data=recognizer.record(source)

    try:

      #recognize the speech
      text = recognizer.recognize_whisper(audio_data)

    except sr.UnknownValueError:
      print("Speech recognition could not understand the audio.")
  return text



In [ ]:
video_url

In [ ]:
output_video_path

In [ ]:
metadata_vid = download_video(video_url, output_video_path)

In [ ]:
metadata_vid

In [ ]:
!mkdir mixed_data

In [ ]:
!mkdir data

In [ ]:
filepath

In [ ]:
output_folder

In [ ]:
video_to_images(filepath,output_folder)

In [ ]:
filepath

In [ ]:
output_audio_path

In [ ]:
video_to_audio(filepath,output_audio_path)

In [ ]:
output_audio_path

In [ ]:
text_data=audio_to_text(output_audio_path)

100%|████████████████████████████████████████| 139M/139M [00:00<00:00, 153MiB/s]


In [ ]:
text_data

In [ ]:
path="/content/data/"

In [ ]:
with open( path+ "text.txt", "w") as file:
        file.write(text_data)
print("Text data saved to file")
file.close()

Text data saved to file


In [ ]:
os.remove(output_audio_path)
print("Audio file removed")

In [ ]:
!pip install -q langchain  vertexai langchain_community chromadb

# **Configure the Vertex AI**

In [ ]:
!pip install --upgrade --user google-cloud-aiplatform

In [ ]:
import getpass, os, requests

if "GCP_PROJECT_ID" not in os.environ:
  os.environ["GCP_PROJECT_ID"] = getpass.getpass("Provide your GCP Project ID")

if "LOCATION" not in os.environ:
  os.environ["LOCATION"] = getpass.getpass("Provide your GCP Location")


In [ ]:
!gcloud config set project {os.getenv("GCP_PROJECT_ID")}

In [ ]:
import sys

# Additional authentication is required for Google Colab
if "google.colab" in sys.modules:
    # Authenticate user to Google Cloud
    from google.colab import auth

    auth.authenticate_user()

In [ ]:
!gcloud auth list

In [ ]:
# Define project information
PROJECT_ID=os.getenv("GCP_PROJECT_ID")
LOCATION=os.getenv("LOCATION")

In [ ]:
# Initialize Vertex AI
import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION)

# **load the text documents**

In [ ]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader("/content/data/text.txt")
documents = loader.load()

# **text data convert into chunk**

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 500)
document = text_splitter.split_documents(text_data)

In [ ]:
from langchain.chat_models import ChatVertexAI
from langchain.llms import VertexAI
from langchain import HuggingFaceHub
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from vertexai.preview.generative_models import (
    GenerationConfig,
    GenerativeModel,
    HarmCategory,
    HarmBlockThreshold,
    Image,
    Part
)

# **Summarizes the text**

In [ ]:
# Prompt
prompt_text = """You are an assistant tasked with summarizing tables and text. \
Give a concise summary of the table or text. Table or text chunk: {element} """
prompt = ChatPromptTemplate.from_template(prompt_text)

# Summary chain
model = VertexAI(
        temperature=0, model_name="gemini-pro")

summarize_chain = {"element": lambda x: x} | prompt | model | StrOutputParser()

In [ ]:
print(type(text_data))

In [ ]:
# Apply to texts
texts = [i.text for i in document]
text_summaries = summarize_chain.batch(texts, {"max_concurrency": 5})

In [ ]:
text_summaries[:50]

In [ ]:
path="/content/mixed_data/"

# **Summarizes the Images**

In [ ]:
import io
import os
import base64
import numpy as np
from PIL import Image
from langchain.chat_models import ChatOpenAI
from langchain.schema.messages import HumanMessage, SystemMessage

def encode_image(image_path):
    ''' Getting the base64 string '''
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

def image_summarize(img_base64,prompt):
    ''' Image summary '''
    chat = ChatVertexAI(model_name="gemini-pro-vision", max_output_tokens=1024)
    msg = chat.invoke(
        [
            HumanMessage(
                content=[
                    {"type": "text", "text":prompt},
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64,{img_base64}"
                        },
                    },
                ]
            )
        ]
    )
    return msg.content

# Store base64 encoded images
img_base64_list = []

# Store image summaries
image_summaries = []
# Prompt
prompt = "Describe the image in detail. Be specific about graphs, such as bar plots."

# Read images, encode to base64 strings
for img_file in sorted(os.listdir(path)):
    if img_file.endswith('.jpg'):
        img_path = os.path.join(path, img_file)
        base64_image = encode_image(img_path)
        img_base64_list.append(base64_image)
        image_summaries.append(image_summarize(base64_image,prompt))

In [ ]:
image_summaries[1]

# **Store the data**

In [ ]:
from vertexai.preview.vision_models import MultiModalEmbeddingModel, Image
embedding = MultiModalEmbeddingModel.from_pretrained("multimodalembedding@001")

In [ ]:
import uuid
from langchain.vectorstores import Chroma
from langchain.storage import InMemoryStore
from langchain.schema.document import Document
from langchain.embeddings import OpenAIEmbeddings
from langchain.retrievers.multi_vector import MultiVectorRetriever

# The vectorstore to use to index the child chunks
vectorstore = Chroma(collection_name="multi_modal_rag",
                     embedding_function=embedding)

# The storage layer for the parent documents
store = InMemoryStore()
id_key = "doc_id"

# The retriever (empty to start)
retriever = MultiVectorRetriever(
    vectorstore=vectorstore,
    docstore=store,
    id_key=id_key,
)

In [ ]:
# Add texts
doc_ids = [str(uuid.uuid4()) for _ in texts]
summary_texts = [
    Document(page_content=s, metadata={id_key: doc_ids[i]})
    for i, s in enumerate(text_summaries)
]
retriever.vectorstore.add_documents(summary_texts)
retriever.docstore.mset(list(zip(doc_ids, texts)))

In [ ]:
len(doc_ids), len(summary_texts)

In [ ]:
# Add image summaries
img_ids = [str(uuid.uuid4()) for _ in img_base64_list]
summary_img = [
    Document(page_content=s, metadata={id_key: img_ids[i]})
    for i, s in enumerate(image_summaries)
]
retriever.vectorstore.add_documents(summary_img)
retriever.docstore.mset(list(zip(img_ids, img_base64_list)))

In [ ]:
len(img_ids), len(summary_img)

# **RAG**

In [ ]:
from operator import itemgetter
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda

def prompt_func(dict):
    format_texts = "\n".join(dict["context"]["texts"])
    return [
        HumanMessage(
            content=[
                {"type": "text", "text": f"""Answer the question based only on the following context, which can include text, tables, and the below image:
Question: {dict["question"]}

Text :
{format_texts}
"""},
                {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{dict['context']['images'][0]}"}},
            ]
        )
    ]

model = ChatOpenAI(temperature=0, model="gpt-4-vision-preview", max_tokens=1024)
# RAG pipeline
chain = (
    {"context": retriever | RunnableLambda(split_image_text_types), "question": RunnablePassthrough()}
    | RunnableLambda(prompt_func)
    | model
    | StrOutputParser()
)

In [ ]:
chain.invoke(
    "What is the change in wild fires from 1993 to 2022?"
)